In [1]:
import sys
from pathlib import Path

In [2]:
module_path = Path.cwd().parents[2]
if module_path not in sys.path:
    sys.path.append(str(module_path))

In [3]:
module_path

PosixPath('/home/marek/Projects/Python/evaluating-student-writing')

In [4]:
import spacy
from spacy.tokens import Doc

/home/marek/mambaforge/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from src.loader import TextLoader
from src.model import DatasetType

In [6]:
loader = TextLoader(dataset_type=DatasetType.V1_WITH_PREDICTIONSTRING)

In [7]:
nlp = spacy.load("models/spacy_resume/model-best/")

<center> <h2> Results for spacy_og/model-best: </h2> </center>

Can be evaluated with:

```bash
python -m spacy evaluate --gpu-id 0 spacy_og/model-best data/NER_test.spacy
```

```
================================== Results ==================================

TOK     -    
NER P   63.22
NER R   73.08
NER F   67.79
SPEED   12164


=============================== NER (per type) ===============================

         P       R       F
DS   63.22   66.78   64.95
DE   63.23   79.40   70.40
```

<center> <h2> Results for spacy_resume/model-best: </h2> </center>

Can be evaluated with:

```bash
python -m spacy evaluate --gpu-id 0 spacy_resume/model-best data/NER_test.spacy
```

```
================================== Results ==================================

TOK     -    
NER P   69.43
NER R   70.28
NER F   69.85
SPEED   11609


=============================== NER (per type) ===============================

         P       R       F
DS   67.87   66.29   67.07
DE   70.89   74.27   72.54
```

In [8]:
nlp.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x7faf673387c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7faf67302cf0>)]

In [64]:
text = loader.load_random_text(purify_discourses=True, purify_text=True)
# text = loader.load_text_with_id("BBB6E0302CFE", purify_discourses=True, purify_text=True)

In [65]:
text.id

'4C84696E7D54'

In [66]:
print(text)

The people of our great nation are told to vote on who they would like to lead and represent them. This is a Democracy and American citizens deserve this right. There have been incidents where the majority of the popular vote from U. S. citizens did not lead to the election of that candidate. This is because of the electoral college. This is the cause of citizens not having the president they asked for. Most American people do not agree with the electoral college. The electoral college is unfair outdated and irrational The Indefensible Electoral College Why even the best laid defenses are wrong Bradford Plumer. The disaster factor is a what the electoral college should really be concerned about. The system allows for many slip ups. The American people should conside themselves lucky that the 2000 fiasco was the biggest election crisis in a century The Indefensible Electoral College Why even the best laid defenses are wrong Bradford Plumer. In 1960 segregationists in the Loisiana legisl

In [67]:
for disc in text.discourses:
    print(disc)

--- 1615835938179 (0 -> 160 | 0 -> 29) - Position ---
The people of our great nation are told to vote on who they would like to lead and represent them. This is a Democracy and American citizens deserve this right.
-----------------------------------------------------
--- 1615835959934 (161 -> 334 | 30 -> 60) - Claim ---
There have been incidents where the majority of the popular vote from U. S. citizens did not lead to the election of that candidate. This is because of the electoral college
-----------------------------------------------------
--- 1615835973009 (336 -> 618 | 61 -> 105) - Evidence ---
This is the cause of citizens not having the president they asked for. Most American people do not agree with the electoral college. The electoral college is unfair outdated and irrational The Indefensible Electoral College Why even the best laid defenses are wrong Bradford Plumer.
---------------------------------------------------------
--- 1615835893365 (619 -> 703 | 106 -> 119) - Clai

In [68]:
word_idx = []
for disc in text.discourses:
    word_idx.extend((disc.predictionstring[0], disc.predictionstring[-1]))

ents = []

DS_token = "B-DS"
DE_token = "B-DE"
use_DS = True
for ind, word in enumerate(text.words):
    if use_DS:
        curr_token = DS_token
    else:
        curr_token = DE_token

    if ind in word_idx:
        ents.append(curr_token)
        use_DS = not use_DS
    else:
        ents.append("O")


In [69]:
manual_doc = Doc(nlp.vocab, text.words, ents=ents)


In [70]:
for token in manual_doc.ents:
    print(token.text, token.label_, token.start_char, token.end_char)

The DS 0 3
right. DE 154 160
There DS 161 166
college. DE 327 335
This DS 336 340
Plumer. DE 611 618
The DS 619 622
about. DE 698 704
The DS 705 708
reality. DE 1490 1498
The DS 1499 1502
about. DE 1543 1549
The DS 1550 1553
them. DE 2198 2203
The DS 2204 2207
vote. DE 2419 2424


In [71]:
doc = nlp(text.text)

In [72]:
for token in doc.ents:
    print(token.text, token.label_, token.start_char, token.end_char)

The DS 0 3
right DE 154 159
There DS 161 166
for DE 402 405
Most DS 407 411
Plumer DE 611 617
The DS 619 622
about DE 698 703
The DS 705 708
candidate DE 1256 1265
Popular DS 1267 1274
about DE 1543 1548
The DS 1550 1553
winner DE 1751 1757
The DS 1759 1762
them DE 2198 2202
The DS 2204 2207
vote DE 2419 2423


In [73]:
spacy.displacy.render(manual_doc, style="ent", jupyter=True)


In [74]:
spacy.displacy.render(doc, style="ent", jupyter=True)